In [3]:
import pandas as pd
import requests
from pytrends.request import TrendReq
import pygsheets
from bs4 import BeautifulSoup as bs
import pprint
from collections import OrderedDict

In [4]:
nation_url = 'https://www.transfermarkt.com/weltmeisterschaft-2018/teilnehmer/pokalwettbewerb/WM18/saison_id/2017'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
nation_headers=[]
nation_info = [[] for i in range(32)]
head = 0
response = requests.get(nation_url, headers=headers)
soup = bs(response.content,"html.parser")

table = soup.find(class_ = 'items')
if head == 0:
    for item in table.find_all('th'):
        nation_headers.append(item.get_text())
    head = 1

table_body = table.find('tbody')
rows = table_body.find_all('tr')
i = 0
for row in rows:
    skip_first = 0
    for element in row.find_all('td'):
        if skip_first == 0:
            skip_first = 1
            continue
        if (str(element.contents[0])[0]) == "<":
            nation_info[i].append(str(element.find('a').contents[0]))
            nation_info[i].append(element.find('a').get('href'))
        else:
            nation_info[i].append(element.contents[0])
    i = i+1

    
nation_headers.pop(0)
nation_headers.insert(1,'Link')
nation_df = pd.DataFrame(nation_info)
nation_df.columns = nation_headers
print(nation_df)
        
    

            Club                                   Link Squad &oslash-Age;  \
0         France     /frankreich/startseite/verein/3377    23         26,0   
1          Spain        /spanien/startseite/verein/3375    23         28,5   
2         Brazil      /brasilien/startseite/verein/3439    23         28,6   
3        Germany    /deutschland/startseite/verein/3262    23         27,2   
4        England        /england/startseite/verein/3299    23         26,1   
5        Belgium        /belgien/startseite/verein/3382    23         27,6   
6      Argentina    /argentinien/startseite/verein/3437    23         29,6   
7       Portugal       /portugal/startseite/verein/3300    23         28,4   
8        Uruguay        /uruguay/startseite/verein/3449    23         28,2   
9        Croatia       /kroatien/startseite/verein/3556    23         27,9   
10       Senegal        /senegal/startseite/verein/3499    23         27,2   
11        Poland          /polen/startseite/verein/3442    23   

In [5]:
nation_players_info = {}

player_name=[]
for nation_link,nation in zip(nation_df['Link'],nation_df['Club']):
    player_info = [[] for i in range(2)]
    player_url='https://www.transfermarkt.com' + nation_link
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}

    response=requests.get(player_url,headers=headers)

    soup = bs(response.content,"html.parser")
    table = soup.find(class_='items')
    every_other = 0
    for item in table.find_all('a',{'class':'spielprofil_tooltip'}):
        player_name.append(str(item.get('title')))
        #print(item.get('title') + ':' + item.get('id'))
        #if every_other == 0:
        player_info[0].append(item.get('title').replace('–','').replace('-','').replace(' ','').replace('\'',''))
        player_info[1].append(item.get('id'))
            #player_info[2].append(nation)
        #    every_other = 1
        #else:
        #    every_other = 0
        
        
    player_info[0]=list(OrderedDict.fromkeys(player_info[0]))
    player_info[1]=list(OrderedDict.fromkeys(player_info[1]))
    #player_info[2] = [nation] * 23
    nation_players_info[nation] = player_info    


#player_info[0]= [x.replace("-","") for x in player_info[0]]
#player_info[0]= [x.replace("\'","") for x in player_info[0]]
print(nation_players_info)

{'France': [['HugoLloris', 'AlphonseAreola', 'SteveMandanda', 'RaphaëlVarane', 'SamuelUmtiti', 'BenjaminPavard', 'PresnelKimpembe', 'AdilRami', 'BenjaminMendy', 'LucasHernández', 'DjibrilSidibé', 'NGoloKanté', 'StevenNzonzi', 'PaulPogba', 'CorentinTolisso', 'BlaiseMatuidi', 'NabilFekir', 'ThomasLemar', 'KylianMbappé', 'OusmaneDembélé', 'FlorianThauvin', 'AntoineGriezmann', 'OlivierGiroud'], ['17965', '120629', '23951', '164770', '126540', '353366', '282041', '51118', '157495', '281963', '161869', '225083', '73734', '122153', '190393', '33923', '203496', '205562', '342229', '288230', '184892', '125781', '82442']], 'Spain': [['DaviddeGea', 'KepaArrizabalaga', 'PepeReina', 'GerardPiqué', 'SergioRamos', 'NachoFernández', 'JordiAlba', 'NachoMonreal', 'DanielCarvajal', 'CésarAzpilicueta', 'ÁlvaroOdriozola', 'SergioBusquets', 'SaúlÑíguez', 'Koke', 'Thiago', 'AndrésIniesta', 'Isco', 'DavidSilva', 'MarcoAsensio', 'LucasVázquez', 'IagoAspas', 'DiegoCosta', 'Rodrigo'], ['59377', '192279', '7825',

In [28]:
print(player_info[0])

#for country in list(nation_players_info):
while(True):
    try:
        data = [[] for i in range(len(player_info[0]))]
        head = 0
        player_headers=["Player"]
        frames={}
        for x in range(len(player_info[0])):

            #print(player_info[0][i] + player_info[1][i] + player_info[2][i])
            #player_url= "https://www.transfermarkt.com/"+ str(player_info[0][x])+ "/leistungsdatendetails/spieler/" + str(player_info[1][x])
            url = "https://www.transfermarkt.com/"+ str(player_info[0][x])+ "/leistungsdatendetails/spieler/" + str(player_info[1][x]) + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
            print(url)
            player_response = requests.get(url, headers = headers )
            player_soup = bs(player_response.content,"html.parser")

            table = player_soup.find(class_='items')

            if head == 0:
                for item in table.find_all('a',{'class':'sort-link'}):
                #print(item.contents[0])
                    if str(type(item.contents[0])) == "<class 'bs4.element.NavigableString'>":
                        player_headers.append(item.contents[0])
                    else:
                        player_headers.append(item.contents[0].get('title'))
                head =1


            table_body = table.find('tbody')
            rows = table_body.find_all('tr')

            for row in rows:
                i=0

                data[i].append(player_info[0][x])
                i=i+1
                for element in row.find_all('td'):

                    if (str(element.contents[0])[0]) == "<":
                        if str(type(element.find('a'))) == "<class 'NoneType'>":
                            1+1
                        else:
                            if str(element.find('a').contents[0])[0]=="<":

                                data[i].append(str(element.find('a').get('href')).strip('/').split('/')[0])
                            else:
                                if str(element.contents[0]) == "-":
                                    data[i].append(0)
                                else:
                                    data[i].append(element.find('a').contents[0])
                    else:
                        if str(element.contents[0]) == "-":
                            data[i].append(0)
                        else:
                            data[i].append(element.contents[0])
                    i=i+1
    except AttributeError as e:
        print(e)
        data[i].append(0)
data.pop(2)
player_headers.pop(3)
player_df = pd.DataFrame(data)
player_df = player_df.transpose()
player_df.columns = player_headers
frames[player_info[0][x]]=player_df

['HugoLloris', 'AlphonseAreola', 'SteveMandanda', 'RaphaëlVarane', 'SamuelUmtiti', 'BenjaminPavard', 'PresnelKimpembe', 'AdilRami', 'BenjaminMendy', 'LucasHernández', 'DjibrilSidibé', 'NGoloKanté', 'StevenNzonzi', 'PaulPogba', 'CorentinTolisso', 'BlaiseMatuidi', 'NabilFekir', 'ThomasLemar', 'KylianMbappé', 'OusmaneDembélé', 'FlorianThauvin', 'AntoineGriezmann', 'OlivierGiroud', 'DaviddeGea', 'KepaArrizabalaga', 'PepeReina', 'GerardPiqué', 'SergioRamos', 'NachoFernández', 'JordiAlba', 'NachoMonreal', 'DanielCarvajal', 'CésarAzpilicueta', 'ÁlvaroOdriozola', 'SergioBusquets', 'SaúlÑíguez', 'Koke', 'Thiago', 'AndrésIniesta', 'Isco', 'DavidSilva', 'MarcoAsensio', 'LucasVázquez', 'IagoAspas', 'DiegoCosta', 'Rodrigo', 'Alisson', 'Ederson', 'Cássio', 'Marquinhos', 'ThiagoSilva', 'Miranda', 'PedroGeromel', 'Marcelo', 'FilipeLuís', 'Danilo', 'Fágner', 'Casemiro', 'Fernandinho', 'Paulinho', 'Fred', 'RenatoAugusto', 'Neymar', 'PhilippeCoutinho', 'Taison', 'DouglasCosta', 'Willian', 'RobertoFirmino

https://www.transfermarkt.com/AlphonseAreola/leistungsdatendetails/spieler/120629/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/SteveMandanda/leistungsdatendetails/spieler/23951/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/RaphaëlVarane/leistungsdatendetails/spieler/164770/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/SamuelUmtiti/leistungsdatendetails/spieler/126540/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/BenjaminPavard/leistungsdatendetails/spieler/353366/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/PresnelKimpembe/leistungsdatendetails/spieler/282041/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/AdilRami/leistungsdatendetails/spieler/51118/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
htt

https://www.transfermarkt.com/Fred/leistungsdatendetails/spieler/191614/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/RenatoAugusto/leistungsdatendetails/spieler/53058/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Neymar/leistungsdatendetails/spieler/68290/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/PhilippeCoutinho/leistungsdatendetails/spieler/80444/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Taison/leistungsdatendetails/spieler/76028/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/DouglasCosta/leistungsdatendetails/spieler/75615/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Willian/leistungsdatendetails/spieler/52769/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Ro

https://www.transfermarkt.com/JanVertonghen/leistungsdatendetails/spieler/43250/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/VincentKompany/leistungsdatendetails/spieler/9594/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/ThomasVermaelen/leistungsdatendetails/spieler/15904/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/DedryckBoyata/leistungsdatendetails/spieler/88262/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/ThomasMeunier/leistungsdatendetails/spieler/100986/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/LeanderDendoncker/leistungsdatendetails/spieler/168157/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/YouriTielemans/leistungsdatendetails/spieler/249565/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plu

https://www.transfermarkt.com/BrunoFernandes/leistungsdatendetails/spieler/240306/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/GonçaloGuedes/leistungsdatendetails/spieler/225122/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/BernardoSilva/leistungsdatendetails/spieler/241641/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/GelsonMartins/leistungsdatendetails/spieler/257448/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/RicardoQuaresma/leistungsdatendetails/spieler/4188/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/CristianoRonaldo/leistungsdatendetails/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/AndréSilva/leistungsdatendetails/spieler/198008/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1


...

https://www.transfermarkt.com/MarkMilligan/leistungsdatendetails/spieler/37372/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/AaronMooy/leistungsdatendetails/spieler/123951/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/MassimoLuongo/leistungsdatendetails/spieler/181687/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/TomRogic/leistungsdatendetails/spieler/213672/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/TimCahill/leistungsdatendetails/spieler/13556/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/DimitriPetratos/leistungsdatendetails/spieler/153882/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/RobbieKruse/leistungsdatendetails/spieler/51972/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.tran

https://www.transfermarkt.com/PedroGallese/leistungsdatendetails/spieler/95172/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/CarlosCáceda/leistungsdatendetails/spieler/94359/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/JoséCarvallo/leistungsdatendetails/spieler/71092/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/ChristianRamos/leistungsdatendetails/spieler/84506/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/AndersonSantamaría/leistungsdatendetails/spieler/142745/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/MiguelAraujo/leistungsdatendetails/spieler/211420/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/AlbertoRodríguez/leistungsdatendetails/spieler/30543/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

https://www.transfermarkt.com/GabrielGómez/leistungsdatendetails/spieler/52028/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/RicardoÁvila/leistungsdatendetails/spieler/440384/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/ArmandoCooper/leistungsdatendetails/spieler/122016/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/JoséLuisRodríguez/leistungsdatendetails/spieler/425028/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
'NoneType' object has no attribute 'find'
https://www.transfermarkt.com/HugoLloris/leistungsdatendetails/spieler/17965/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/AlphonseAreola/leistungsdatendetails/spieler/120629/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/SteveMandanda/leistungsdatendetails/spieler/23951/saison//verein/0/liga/

https://www.transfermarkt.com/FilipeLuís/leistungsdatendetails/spieler/21725/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Danilo/leistungsdatendetails/spieler/145707/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Fágner/leistungsdatendetails/spieler/47592/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Casemiro/leistungsdatendetails/spieler/16306/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Fernandinho/leistungsdatendetails/spieler/26267/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Paulinho/leistungsdatendetails/spieler/57229/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/Fred/leistungsdatendetails/spieler/191614/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/RenatoAugust

KeyboardInterrupt: 

In [6]:
#print(nation_players_info['France'][1][0])
#print(list(nation_players_info.keys()))
player_stats={}
player_headers=["Nation","Player","Position"]
header_mark=0
posit_headers={}
data =[[] for i in range(21)]
for nation in list(nation_players_info.keys()):
    players = (nation_players_info[nation][0])
    ids = (nation_players_info[nation][1])

    try:
        for i in range(len(players)):
            url = "https://www.transfermarkt.com/"+ str(players[i])+ "/leistungsdatendetails/spieler/" + str(ids[i]) + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
            print("Player : " + str(players[i]) + " from " + nation) 
            player_response = requests.get(url, headers = headers )
            player_soup = bs(player_response.content,"html.parser")

            head = player_soup.find(text="Position:")
            position = (head.findNext('span').contents[0]).strip()
            
            table = player_soup.find(class_='items')
            
            if table is None:
                print("Table is none")
                continue

            
            player_headers=["Nation","Player","Position"]    
            for item in table.find_all('a',{'class':'sort-link'}):
            #print(item.contents[0])
                
                if str(type(item.contents[0])) == "<class 'bs4.element.NavigableString'>":
                    player_headers.append(item.contents[0])
                else:
                    player_headers.append(item.contents[0].get('title'))
            player_headers.pop(4)
            posit_headers[position] = player_headers
                
    
            table_body = table.find('tbody')
            rows = table_body.find_all('tr')
            
            for row in rows:
                j = 0
                data[j].append(nation)
                j=j+1
                data[j].append(str(players[i]))
                j=j+1
                data[j].append(str(position))
                j=j+1
                for element in row.find_all('td'):
                    if (str(element.contents[0])[0]) == "<":
                        if str(type(element.find('a'))) == "<class 'NoneType'>":
                            1+1
                        else:
                            if str(element.find('a').contents[0])[0] == "<":
                                data[j].append(str(element.find('a').get('href')).strip('/').split('/')[0])
                            else:
                                if str(element.contents[0]) == "-":
                                    data[j].append(0)
                                else:
                                    data[j].append(element.find('a').contents[0])
                    else:
                        if str(element.contents[0]) == "-":
                            data[j].append(0)
                        else:
                            data[j].append(element.contents[0])
                    j=j+1
                
    except AttributeError as e:
        print(e)
        
print("DONE")





Player : HugoLloris from France
Player : AlphonseAreola from France
Player : SteveMandanda from France
Player : RaphaëlVarane from France
Player : SamuelUmtiti from France
Player : BenjaminPavard from France
Player : PresnelKimpembe from France
Player : AdilRami from France
Player : BenjaminMendy from France
Player : LucasHernández from France
Player : DjibrilSidibé from France
Player : NGoloKanté from France
Player : StevenNzonzi from France
Player : PaulPogba from France
Player : CorentinTolisso from France
Player : BlaiseMatuidi from France
Player : NabilFekir from France
Player : ThomasLemar from France
Player : KylianMbappé from France
Player : OusmaneDembélé from France
Player : FlorianThauvin from France
Player : AntoineGriezmann from France
Player : OlivierGiroud from France
Player : DaviddeGea from Spain
Player : KepaArrizabalaga from Spain
Player : PepeReina from Spain
Player : GerardPiqué from Spain
Player : SergioRamos from Spain
Player : NachoFernández from Spain
Player : 

Player : SaliouCiss from Senegal
Player : YoussoufSabaly from Senegal
Player : LamineGassama from Senegal
Player : MoussaWagué from Senegal
Player : CheikhouKouyaté from Senegal
Player : AlfredNDiaye from Senegal
Player : IdrissaGueye from Senegal
Player : BadouNdiaye from Senegal
Player : CheikhNDoye from Senegal
Player : SadioMané from Senegal
Player : KeitaBaldé from Senegal
Player : IsmaïlaSarr from Senegal
Player : MBayeNiang from Senegal
Player : MameDiouf from Senegal
Player : MoussaKonaté from Senegal
Player : DiafraSakho from Senegal
Player : MoussaSow from Senegal
Player : WojciechSzczesny from Poland
Player : LukaszFabianski from Poland
Player : BartoszBialkowski from Poland
Player : KamilGlik from Poland
Player : JanBednarek from Poland
Player : MichalPazdan from Poland
Player : ThiagoCionek from Poland
Player : BartoszBereszynski from Poland
Player : LukaszPiszczek from Poland
Player : ArturJedrzejczyk from Poland
Player : GrzegorzKrychowiak from Poland
Player : JacekGoral

Player : SofyanAmrabat from Morocco
Player : KarimElAhmadi from Morocco
Player : NabilDirar from Morocco
Player : HakimZiyech from Morocco
Player : AmineHarit from Morocco
Player : YounèsBelhanda from Morocco
Player : FayçalFajr from Morocco
Player : MoubarakBoussoufa from Morocco
Player : MehdiCarcelaGonzález from Morocco
Player : NordinAmrabat from Morocco
Player : YoussefEnNesyri from Morocco
Player : AyoubElKaabi from Morocco
Player : KhalidBoutaïb from Morocco
Player : AzizBouhaddouz from Morocco
Player : KarlJohanJohnsson from Sweden
Player : KristofferNordfeldt from Sweden
Player : RobinOlsen from Sweden
Player : VictorLindelöf from Sweden
Player : PontusJansson from Sweden
Player : AndreasGranqvist from Sweden
Player : FilipHelander from Sweden
Player : LudwigAugustinsson from Sweden
Player : MartinOlsson from Sweden
Player : MikaelLustig from Sweden
Player : EmilKrafth from Sweden
Player : GustavSvensson from Sweden
Player : OscarHiljemark from Sweden
Player : AlbinEkdal from 

Player : YasirAlShahrani from Saudi Arabia
Player : MansourAlHarbi from Saudi Arabia
Player : MohammedAlBurayk from Saudi Arabia
Player : AbdullahAteef from Saudi Arabia
Player : AbdulmalekAlKhaibri from Saudi Arabia
Player : AbdullahAlKhaibari from Saudi Arabia
Player : SalmanAlFaraj from Saudi Arabia
Player : HusseinAlMoghawi from Saudi Arabia
Player : TaisirAlJassim from Saudi Arabia
Player : MohamedKanno from Saudi Arabia
Player : HattanBahbir from Saudi Arabia
Player : YahiaAlShehri from Saudi Arabia
Player : FahadAlMuwallad from Saudi Arabia
Player : SalemAlDawsari from Saudi Arabia
Player : MuhannadAsiri from Saudi Arabia
Player : MohammedAlSahlawi from Saudi Arabia
Player : JaimePenedo from Panama
Player : JoséCalderón from Panama
Player : ÁlexRodríguez from Panama
Player : RománTorres from Panama
Player : AdolfoMachado from Panama
Player : HároldCummings from Panama
Player : FidelEscobar from Panama
Player : FelipeBaloy from Panama
Player : LuisOvalle from Panama
Player : Éric

In [24]:

df = pd.DataFrame(data)
df = df.transpose()
print(posit_headers.keys())
print(posit_headers['Defensive Midfield'])
print(df[df[2] == "Keeper"])

dict_keys(['Keeper', 'Centre-Back', 'Left-Back', 'Right-Back', 'Defensive Midfield', 'Central Midfield', 'Attacking Midfield', 'Left Wing', 'Right Wing', 'Centre-Forward', 'Secondary Striker', 'Left Midfield', 'Right Midfield'])
['Nation', 'Player', 'Position', 'Season', 'reihenfolge ASC, saison_id DESC, verein_id ASC', 'Club(s)', 'Squad', 'Apps', 'PPM', 'Goals', 'Assists', 'own goals', 'Substituted on', 'Substituted off', 'Yellow cards', 'Yellow/red cards', 'Red cards', 'Penalty goals', 'Minutes per goal', 'Minutes played']
                 0                  1       2      3     4   \
0            France         HugoLloris  Keeper  17/18         
1            France         HugoLloris  Keeper  17/18         
2            France         HugoLloris  Keeper  17/18         
3            France         HugoLloris  Keeper  16/17         
4            France         HugoLloris  Keeper  16/17         
5            France         HugoLloris  Keeper  16/17         
6            France         

In [9]:
#print(df.columns[[4,20]])
position_agg = list(df[2].unique())

In [25]:
posit_players={}
print(len(df.columns))
for posit in position_agg:
    temp_df = df[df[2]==posit]
    #posit_headers[posit].pop(4)
    print(posit_headers[posit])
    #posit_players[posit] = temp_df
    if (posit == "Keeper"):
        posit_players[posit]= temp_df.drop(df.columns[[4,20]],axis = 1)
    else:
        posit_players[posit]= temp_df.drop(df.columns[[4]],axis = 1)
    
    print(posit_players[posit])
    posit_players[posit].columns = posit_headers[posit]

21
['Nation', 'Player', 'Position', 'Season', 'reihenfolge ASC, saison_id DESC, verein_id ASC', 'Club(s)', 'Squad', 'Apps', 'PPM', 'Goals', 'own goals', 'Substituted on', 'Substituted off', 'Yellow cards', 'Yellow/red cards', 'Red cards', 'Conceded goals', 'Matches without conceded goals', 'Minutes played']
                 0                  1       2      3   \
0            France         HugoLloris  Keeper  17/18   
1            France         HugoLloris  Keeper  17/18   
2            France         HugoLloris  Keeper  17/18   
3            France         HugoLloris  Keeper  16/17   
4            France         HugoLloris  Keeper  16/17   
5            France         HugoLloris  Keeper  16/17   
6            France         HugoLloris  Keeper  16/17   
7            France         HugoLloris  Keeper  15/16   
8            France         HugoLloris  Keeper  15/16   
9            France         HugoLloris  Keeper  15/16   
10           France         HugoLloris  Keeper  15/16   
11     

[186 rows x 20 columns]
['Nation', 'Player', 'Position', 'Season', 'reihenfolge ASC, saison_id DESC, verein_id ASC', 'Club(s)', 'Squad', 'Apps', 'PPM', 'Goals', 'Assists', 'own goals', 'Substituted on', 'Substituted off', 'Yellow cards', 'Yellow/red cards', 'Red cards', 'Penalty goals', 'Minutes per goal', 'Minutes played']
               0              1              2      3   \
3513      England    AshleyYoung  Left Midfield  17/18   
3514      England    AshleyYoung  Left Midfield  17/18   
3515      England    AshleyYoung  Left Midfield  17/18   
3516      England    AshleyYoung  Left Midfield  17/18   
3517      England    AshleyYoung  Left Midfield  17/18   
3518      England    AshleyYoung  Left Midfield  16/17   
3519      England    AshleyYoung  Left Midfield  16/17   
3520      England    AshleyYoung  Left Midfield  16/17   
3521      England    AshleyYoung  Left Midfield  16/17   
3522      England    AshleyYoung  Left Midfield  15/16   
3523      England    AshleyYoung  Le

In [32]:
#print((posit_players['Centre-Forward']))

for item in posit_players:
    posit_players[item].to_csv(item+".csv",sep=',')